In [1]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics
using Printf

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [2]:
# model definition
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
# render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file_combo = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alpha_seabotix_nojoints.urdf")
mechanism_alpha = parse_urdf(urdf_file_combo; root_joint_type=SPQuatFloating(), gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file_combo)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file_combo), vis[:alpha])
# render(mvis_alpha)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, SPQuatFloating{Float64}}}, Vector{Joint{Float64, Revolute{Float64}}}}, 2}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat/alpha at http://127.0.0.1:8700, 30)

In [3]:
# Name the joints and bodies of the mechanism
vehicle_joint, base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
~, vehicle_body, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)
num_virtual_links = 0

body_frame = default_frame(vehicle_body)
shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# setelement!(mvis_alpha, shoulder_frame)

CartesianFrame3D: "world" (id = 0)

In [4]:
cur_state = MechanismState(mechanism_alpha)
zero!(cur_state)
set_configuration!(cur_state, vehicle_joint, [0., 0., 0.5, 1., 0., 0.])

for jointid in cur_state.treejointids
    println("new joint!")
end

new joint!
new joint!
new joint!
new joint!
new joint!


In [15]:
frame_names_cob = ["vehicle_cob", "shoulder_cob", "ua_cob", "elbow_cob", "wrist_cob"]
frame_names_com = ["vehicle_com", "shoulder_com", "ua_com", "elbow_com", "wrist_com"]
cob_vecs = [SVector{3, Float64}([0.0, 0.0, -.01]), SVector{3, Float64}([-0.001, -0.003, .032]), SVector{3, Float64}([0.073, 0.0, -0.002]), SVector{3, Float64}([0.015, -0.012, -.003]), SVector{3, Float64}([0.0, 0.0, -.098])]
com_vecs = [SVector{3, Float64}([0.0, 0.0, -0.06]), SVector{3, Float64}([0.005, -.001, 0.016]), SVector{3, Float64}([0.073, 0.0, 0.0]), SVector{3, Float64}([0.017, -0.026, -0.002]), SVector{3, Float64}([0.0, 0.003, -.098])]
cob_frames = []
com_frames = []
for i in 1:5
    bod = bodies(mechanism_alpha)[i+1]
    frame_cob = CartesianFrame3D(frame_names_cob[i])
    frame_com = CartesianFrame3D(frame_names_com[i])
    cob_vec = cob_vecs[i]
    com_vec = com_vecs[i]
    cob_transform = Transform3D(frame_cob, default_frame(bod), cob_vec)
    com_transform = Transform3D(frame_com, default_frame(bod), com_vec)
    # setelement!(mvis_alpha, default_frame(bod))
    if !(is_fixed_to_body(bod, frame_cob))
        add_frame!(bod, cob_transform)
        push!(cob_frames, frame_cob)
        if i == 3
            setelement!(mvis_alpha, frame_cob)    # visualizes COB frames in MeshCat
        end
    end
    if !(is_fixed_to_body(bod, frame_com))
        add_frame!(bod, com_transform)
        push!(com_frames, frame_com)
        # setelement!(mvis_alpha, frame_com)    # visualizes COM frames in MeshCat
    end
end

# alphabase_com_wrt_linkframe = SVector{3, Float64}([-0.075, -0.006, -.003])
alphabase_com_wrt_linkframe = SVector{3, Float64}([-0.075, -0.0, -.0])
linkframe_wrt_vehframe = translation(frame_definitions(vehicle_body)[38])
# IF THE ARM IS ROTATED THIS HAS TO CHANGE!!!!
alphabase_com_wrt_vehframe = alphabase_com_wrt_linkframe + linkframe_wrt_vehframe
alphabase_com_frame = CartesianFrame3D("armbase_com_cob")
com_transform = Transform3D(alphabase_com_frame, default_frame(vehicle_body), alphabase_com_wrt_vehframe)

if !(is_fixed_to_body(vehicle_body, alphabase_com_frame))
    add_frame!(vehicle_body, com_transform)
    push!(cob_frames, alphabase_com_frame)
    push!(com_frames, alphabase_com_frame)
    # setelement!(mvis_alpha, alphabase_com_frame)
end
print("THIS SHOULD SAY after arm_to_vehicle: ")
println(frame_definitions(vehicle_body)[38].from)

# println(alphabase_com_wrt_vehframe)

println("URDFs parsed. Visualizers ready. \n")

THIS SHOULD SAY after arm_to_vehicle: after_arm_to_vehicle
URDFs parsed. Visualizers ready. 



In [16]:
# buoyancy force setup
# ------------------------------------------------------------------------
#                           BUOYANCY SETUP
# ------------------------------------------------------------------------
# f = rho * g * V
# f = 997 (kg/m^3) * 9.81 (m/s^2) * V_in_L *.001 (m^3) = kg m / s^2
# One time setup of buoyancy forces
rho = 997
volumes = [22.2/(.001*rho), .018, .203, .025, .155, .202] # vehicle, shoulder, ua, elbow, wrist, armbase
buoy_force_mags = volumes * 997 * 9.81 * .001
buoy_lin_forces = []
for mag in buoy_force_mags
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, mag])
    push!(buoy_lin_forces, lin_force)
end
print(buoy_lin_forces)

masses = [22.2, .194, .429, .115, .333, .341]
grav_forces = masses*9.81
grav_lin_forces = []
for f_g in grav_forces
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, -f_g])
    push!(grav_lin_forces, lin_force)
end

drag_link1 = [0.26 0.26 0.3]*rho
drag_link2 = [0.3 1.6 1.6]*rho
drag_link3 = [0.26 0.3 0.26]*rho
drag_link4 = [1.8 1.8 0.3]*rho
link_drag_coeffs = [drag_link1, drag_link2, drag_link3, drag_link4]
# println(spatial_inertia(vehicle_body))
# println(spatial_inertia(shoulder_body))
# println(spatial_inertia(upper_arm_body))
# println(spatial_inertia(elbow_body))
# println(spatial_inertia(wrist_body))
# println(mass(mechanism_alpha))

Any[FreeVector3D in "world": [0.0, 0.0, 217.782], FreeVector3D in "world": [0.0, 0.0, 0.17605026], FreeVector3D in "world": [0.0, 0.0, 1.9854557100000003], FreeVector3D in "world": [0.0, 0.0, 0.24451425000000004], FreeVector3D in "world": [0.0, 0.0, 1.5159883500000002], FreeVector3D in "world": [0.0, 0.0, 1.97567514]]

4-element Vector{Matrix{Float64}}:
 [259.22 259.22 299.09999999999997]
 [299.09999999999997 1595.2 1595.2]
 [259.22 299.09999999999997 259.22]
 [1794.6000000000001 1794.6000000000001 299.09999999999997]

In [146]:
# Hydrodynamics calculator function
# ------------------------------------------------------------------------
#                HYDRODYNAMICS (Grav & Buoy) CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict{BodyID, Wrench{Float64}}, t, state_alpha::MechanismState)
    buoy_wrenches = []
    grav_wrenches = []
    names = ["cob1", "cob2", "cob3", "cob4", "cob5"]
    # Iterate through each body 
    for i in 1:5
        # Get the body
        bod = bodies(mechanism_alpha)[i+1]
        # Get default frame of the body
        body_default_frame = default_frame(bod)

        # -------- Calculate Buoyancy Wrench-------
        # Get transform between the defualt frame and the center of buoyancy
        # TODO: don't use fixed_transform because it's bad for computation time
        def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
        # Transform buoyancy force vector to the body's default frame (rotation only)
        buoy_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
        # Make the wrench: the buoyancy force through a point, the center of buoyancy.
        buoy_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_cob))), buoy_force_trans)
        push!(buoy_wrenches, buoy_wrench)

        # -------- Calculate Gravity Wrench -------
        def_to_com = fixed_transform(bod, body_default_frame, com_frames[i])
        grav_force_trans = transform(state_alpha, grav_lin_forces[i], body_default_frame)
        # Make the wrench: the buoyancy force through a point, the center of buoyancy.
        grav_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_com))), grav_force_trans)
        # Add wrench to buoy_wrenches
        push!(grav_wrenches, grav_wrench)

        # Add the buoyancy wrench and grav wrench together
        wrench = buoy_wrench + grav_wrench
        # println(wrench)
        # Visualize location of center of buoyancy
        # setelement!(mvis_alpha, Point3D(body_default_frame, translation(inv(def_to_cob))), 0.02, "name")

        # ----- Special calculaitons for the vehicle -----
        if i == 1
            # ----- Grav/buoy for arm base link ----- 
            def_to_armbase_cob = fixed_transform(bod, body_default_frame, cob_frames[6])
            def_to_armbase_com = fixed_transform(bod, body_default_frame, com_frames[6])
            buoy_force_trans_armbase = transform(state_alpha, buoy_lin_forces[6], body_default_frame)
            grav_force_trans_armbase = transform(state_alpha, grav_lin_forces[6], body_default_frame)
            buoy_wrench_arm = Wrench(Point3D(body_default_frame, translation(inv(def_to_armbase_cob))), buoy_force_trans_armbase)
            grav_wrench_arm = Wrench(Point3D(body_default_frame, translation(inv(def_to_armbase_com))), grav_force_trans_armbase)
            wrench = wrench + buoy_wrench_arm + grav_wrench_arm
            # println("armbase gravity wrench in vehicle frame")
            # println(grav_wrench_arm)
            # setelement!(mvis_alpha, Point3D(body_default_frame, translation(inv(def_to_armbase_com))), 0.02, "armbase_com")
            # println("Wrench without drag:")
            # println(wrench)

            # ----- Drag of the vehicle -----
            vel=velocity(state_alpha, joints(state_alpha.mechanism)[1])
            d_lin_coeffs = [4.5, 8.0, 2.7, 3.4, 4.6, 52.9]
            d_nonlin_coeffs = [11.4, 20.0, 13.5, 34.4, 65.9, 132.3]
            tau_d = d_lin_coeffs .* abs.(vel) .+ d_nonlin_coeffs .* vel .* abs.(vel)
            drag_wrench = Wrench(body_default_frame, tau_d[1:3], tau_d[4:6])  
            wrench = wrench - drag_wrench
            # println("Wrench drag:")
            # println(drag_wrench)
        # ----- Drag on the links (quadratic only) ----
        else
            twist_world = twist_wrt_world(state_alpha, bod)
            root_transform = transform_to_root(state_alpha, bod)
            # COB_point = Point3D(body_default_frame, translation(inv(def_to_cob)))
            twist_body = transform(twist_world, inv(root_transform))
            cob_vel = point_velocity(twist_body, Point3D(body_default_frame, translation(inv(def_to_cob))))
            # @show(i)

            F_d = transpose(link_drag_coeffs[i-1]) .* abs.(cob_vel.v) .* cob_vel.v
            # println([@printf(" %5.2f", x) for x in twist_body.linear])
            # println([@printf(" %5.2f", x) for x in F_d])
            # Wrench(frame, angular, linear)
            drag_wrench_at_cob = Wrench(cob_frames[i], [0.0, 0.0, 0.0], [F_d[1], F_d[2], F_d[3]])
            drag_wrench_at_default = transform(drag_wrench_at_cob, inv(def_to_cob))
            wrench = wrench - drag_wrench_at_default
        end
        # Transform the wrench to the root frame and assign it to the body
        hydro_wrenches[BodyID(bod)] = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    end
end;


In [185]:
# Controller function
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState, hydro_wrenches::Dict{BodyID, Wrench{Float64}})
    # Get buoyancy buoyancy forces 
    # hydro_wrenches = Dict{BodyID, Wrench{Float64}}()
    # hydro_calc!(hydro_wrenches, t, state_alpha) 
    # Calculate inverse dynamics of alpha arm (including buoyancy)
    # tau = inverse_dynamics(state_alpha, des_acc, hydro_wrenches)
    # println(des_acc)

    # println("Control Tau:")
    # println(tau)

    # Set torques accordingly (controller)
    
    # fill!(torques[velocity_range(state_alpha, vehicle_joint)], 0.0)
    # torques[velocity_range(state_alpha, vehicle_joint)] .= [0.0, 0.0, 0.0, 50.0, 0.0, 0.0]
    torques[velocity_range(state_alpha, vehicle_joint)] .= [0.0, 0.0, 0.0, 0.0, 0.0, 7.5]
    # torques[vehicle_joint][2] = 0.0
    torques[velocity_range(state_alpha, shoulder_joint)] .= 0.00
    torques[velocity_range(state_alpha, base_joint)] .= 0.00
    torques[velocity_range(state_alpha, elbow_joint)] .= 0.0
    torques[velocity_range(state_alpha, wrist_joint)] .= 0.0

    # torques[velocity_range(state_alpha, vehicle_joint)] = tau[vehicle_joint][1:6]
    # torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    # torques[velocity_range(state_alpha, shoulder_joint)] .= tau[shoulder_joint][1]
    # torques[velocity_range(state_alpha, elbow_joint)] .= tau[elbow_joint][1]
    # torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]
    # println(torques[vehicle_joint])
end;

In [186]:
# initial conditions
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
set_configuration!(state_alpha, vehicle_joint, [0., 0., 0., 0., 0., 0.])
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Define desired accelerations
des_acc = similar(velocity(state_alpha))
fill!(des_acc[vehicle_joint], 0.0)
des_acc[vehicle_joint][1] = 1.0
des_acc[base_joint][1] = 0.0
des_acc[shoulder_joint][1] = 0.0
des_acc[elbow_joint][1] = 0.0
des_acc[wrist_joint][1] = 0.0


0.0

In [187]:
# simulation
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 10
ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 5e-3)
# ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

render(mvis_alpha)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("alpha" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("vehicle_base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_base_to_sf_fan" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x74, 0x6f, 0x5f, 0x73, 0x66, 0x5f, 0x66, 0x61, 0x6e], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_12" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "after_base_to_sa_fan" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x74, 0x6f, 0x5f, 0x73, 0x61, 0x5f, 0x66, 0x61, 0x6e], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_16" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x36], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x36], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "r5m_shoulder_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x6c, 0x64, 0x65, 0x72, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_r5m_joint5" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x35, 0x6d, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x35], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_22" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "r5m_upper_arm_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x5f, 0x61, 0x72, 0x6d, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_r5m_joint4" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x35, 0x6d, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_23" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79,

In [190]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])

MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 0.5)
println(last(qs))
println(last(vs))

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")